# Ingeniería de Características para Mantenimiento Predictivo
## Proyecto: Predicción de Fallas en Moto-Compresores - Oil & Gas

### 🎯 Objetivo del Notebook

Este notebook constituye la **fase crítica** de transformación de datos donde convertimos las series temporales limpias en un dataset enriquecido y etiquetado, optimizado para el entrenamiento de modelos de Machine Learning. Nuestro objetivo principal es **predecir fallas en moto-compresores con 7 días de antelación**, una ventana temporal que permite la planificación efectiva de mantenimientos preventivos en el sector Oil & Gas.

### 📋 Tareas Principales

1. **Carga y Validación de Datos**: Integrar el dataset preprocesado con el historial de eventos
2. **Ingeniería de Características Temporales**: Crear features que capturen la dinámica del deterioro
3. **Características Avanzadas**: Implementar features de tasas de cambio, frecuencia y detección de anomalías
4. **Etiquetado de Fallas**: Crear la variable objetivo basada en ventanas de pre-falla de 7 días
5. **Validación y Preparación Final**: Garantizar calidad de datos para modelado

In [3]:
# Importación de librerías esenciales para ingeniería de características
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import warnings

# Librerías especializadas para análisis de señales y anomalías
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Configuración del entorno
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías importadas exitosamente")
print(f"📊 Versión de pandas: {pd.__version__}")
print(f"🔢 Versión de numpy: {np.__version__}")

✅ Librerías importadas exitosamente
📊 Versión de pandas: 2.2.2
🔢 Versión de numpy: 2.0.2


In [4]:
import os

print("📂 Creando estructura de directorios...")
os.makedirs('data/processed', exist_ok=True)
os.makedirs('eventos', exist_ok=True)
print("✅ Carpetas listas: 'data/processed' y 'eventos'")

📂 Creando estructura de directorios...
✅ Carpetas listas: 'data/processed' y 'eventos'


In [5]:
# Configuración de rutas de datos con validación de existencia
# Esta configuración garantiza la reproducibilidad del pipeline

# Directorio base del proyecto
base_dir = Path('.')

# Rutas específicas para datos procesados y eventos
ruta_processed = base_dir / 'data' / 'processed'
ruta_eventos = base_dir / 'eventos'

# Archivos específicos requeridos
archivo_timeseries = ruta_processed / 'timeseries_data.parquet'
archivo_historial = ruta_eventos / 'Historial C1 RGD.xlsx'

# Validación crítica de existencia de archivos
print("📁 Validación de rutas y archivos:")
print(f"   Datos procesados: {ruta_processed} - {'✅ Existe' if ruta_processed.exists() else '❌ No existe'}")
print(f"   Eventos: {ruta_eventos} - {'✅ Existe' if ruta_eventos.exists() else '❌ No existe'}")
print(f"   Timeseries: {archivo_timeseries} - {'✅ Existe' if archivo_timeseries.exists() else '❌ No existe'}")
print(f"   Historial: {archivo_historial} - {'✅ Existe' if archivo_historial.exists() else '❌ No existe'}")

# Verificación crítica - detener ejecución si faltan archivos esenciales
archivos_requeridos = [archivo_timeseries, archivo_historial]
archivos_faltantes = [arch for arch in archivos_requeridos if not arch.exists()]

if archivos_faltantes:
    print(f"\n❌ ERROR CRÍTICO: Faltan archivos esenciales:")
    for archivo in archivos_faltantes:
        print(f"   - {archivo}")
    raise FileNotFoundError("No se pueden continuar sin los archivos de datos requeridos")
else:
    print("\n✅ Todos los archivos requeridos están disponibles")

📁 Validación de rutas y archivos:
   Datos procesados: data/processed - ✅ Existe
   Eventos: eventos - ✅ Existe
   Timeseries: data/processed/timeseries_data.parquet - ✅ Existe
   Historial: eventos/Historial C1 RGD.xlsx - ✅ Existe

✅ Todos los archivos requeridos están disponibles


## 1. 📂 Carga y Validación de Datos

### 🔄 Proceso de Carga Inteligente

En esta fase crítica, cargaremos tanto el **dataset de series temporales procesado** como el **historial de eventos de mantenimiento**. La calidad de este proceso determina directamente la efectividad de nuestro modelo predictivo.

El dataset de series temporales contiene las mediciones continuas de sensores del moto-compresor, ya limpias y preprocesadas. El historial de eventos proporciona las fechas exactas de las fallas históricas, información esencial para crear nuestras etiquetas de entrenamiento.

In [6]:
# Carga optimizada del dataset principal de series temporales
print("⚡ Cargando dataset principal de series temporales...")

try:
    # Cargar el dataset principal con optimizaciones de memoria
    df = pd.read_parquet(archivo_timeseries, engine='pyarrow')
    # Convertir la columna 'hora' en DatetimeIndex
    if 'hora' in df.columns:
        df['hora'] = pd.to_datetime(df['hora'])
        df.set_index('hora', inplace=True)
    print(df.head(2))
    print(df.info())

    # Ordenar el índice del DataFrame por si acaso no está en orden cronológico
    print("🔧 Ordenando el DataFrame por el índice de tiempo...")
    df = df.sort_index()

    # 3. (Opcional pero recomendado) Verifica que el índice ya está ordenado
    if df.index.is_monotonic_increasing:
        print("✅ El índice está ordenado correctamente.")
    else:
        # Esto no debería pasar después de sort_index(), pero es una buena comprobación
        print("⚠️ El índice AÚN no está ordenado. Revisa la fuente de datos.")

    # --- LIMPIEZA DEL DATAFRAME PRINCIPAL ---
    if not df.index.is_unique:
        duplicados_encontrados = df.index.duplicated().sum()
        print(f"⚠️  Índice principal con {duplicados_encontrados} duplicados. Limpiando...")
        df = df[~df.index.duplicated(keep='first')]
        print(f"✅ Índice principal limpio. Total filas ahora: {len(df)}")
    else:
        print("✅ Índice principal ya es único.")

    # Validaciones críticas inmediatas
    if df.empty:
        raise ValueError("El dataset cargado está vacío")

    """ if not isinstance(df.index, pd.DatetimeIndex):
        print("⚠️  Convirtiendo índice a DatetimeIndex")
        df.index = pd.to_datetime(df.index) """

    # Información básica del dataset
    print(f"✅ Dataset principal cargado exitosamente")
    print(f"   📊 Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    print(f"   📅 Período temporal: {df.index.min()} → {df.index.max()}")
    print(f"   ⏱️  Frecuencia detectada: {pd.infer_freq(df.index) or 'Variable/No detectada'}")
    print(f"   💾 Uso de memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    # Análisis de calidad de datos
    completitud_promedio = (df.count().sum() / (df.shape[0] * df.shape[1])) * 100
    print(f"   📈 Completitud promedio: {completitud_promedio:.1f}%")

    # Verificar tipos de datos
    tipos_datos = df.dtypes.value_counts()
    print(f"   🔢 Tipos de datos: {dict(tipos_datos)}")

except Exception as e:
    print(f"❌ Error crítico al cargar el dataset principal: {str(e)}")
    raise

⚡ Cargando dataset principal de series temporales...
                     pres_aceite_comp          rpm  presion_aceite_motor  presion_agua  presion_carter  temp_aceite_motor  temp_agua_motor  temp_mult_adm_izq
hora                                                                                                                                                         
2023-01-01 00:00:00         60.995666  1099.724817             57.188761     15.238856        0.540845          89.621252        86.009180          57.552369
2023-01-01 01:00:00         60.244113  1100.752074             55.039625     15.134238        0.549441          90.526367        84.226618          60.181223
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20401 entries, 2023-01-01 00:00:00 to 2025-04-30 00:00:00
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pres_aceite_comp      20401 non-null  float64
 1   rpm           

In [7]:
# Carga y procesamiento del historial de eventos - VERSIÓN BLINDADA
print("\n🔄 Cargando historial de eventos de mantenimiento (FORZADO)...")

def cargar_historial_eventos_blindado(archivo_historial):
    try:
        # 1. Cargar SIN cabecera para no perder la primera fila de datos
        df_raw = pd.read_excel(archivo_historial, header=None, engine='openpyxl')

        print(f"📊 Dimensiones crudas: {df_raw.shape}")

        # 2. Identificar columnas clave automáticamente
        # Buscamos columnas que parezcan fechas
        col_fecha = None
        col_desc = None

        for col in df_raw.columns:
            # Verificar si la columna es tipo fecha o parece fecha
            try:
                temp_date = pd.to_datetime(df_raw[col], errors='coerce')
                validas = temp_date.notna().sum()
                if validas > 0.5 * len(df_raw): # Si más del 50% son fechas
                    col_fecha = col
                    print(f"   📅 Columna de fecha detectada: {col} ({validas} fechas válidas)")
                    break
            except:
                continue

        # Si no detectamos fecha, asumimos la primera columna
        if col_fecha is None:
            col_fecha = 0
            print("   ⚠️ No se detectó columna de fecha, asumiendo columna 0")

        # 3. Renombrar columnas manualmente (Estándar)
        # Asumimos estructura: [FECHA, DESCRIPCIÓN, EQUIPO] o similar
        nuevas_columnas = {
            col_fecha: 'fecha_evento',
            col_fecha + 1: 'descripcion_falla' if (col_fecha + 1) in df_raw.columns else 'col_1',
            col_fecha + 2: 'equipo' if (col_fecha + 2) in df_raw.columns else 'col_2'
        }

        df_eventos = df_raw.rename(columns=nuevas_columnas)

        # 4. Limpieza final
        df_eventos['fecha_evento'] = pd.to_datetime(df_eventos['fecha_evento'], errors='coerce')
        df_eventos = df_eventos.dropna(subset=['fecha_evento'])

        print(f"✅ Historial procesado exitosamente")
        print(f"   📅 Eventos válidos: {len(df_eventos)}")
        print(f"   📋 Columnas: {list(df_eventos.columns)}")
        print(f"   🔍 Ejemplo: {df_eventos.iloc[0]['fecha_evento']} - {df_eventos.iloc[0].get('descripcion_falla', 'N/A')}")

        return df_eventos

    except Exception as e:
        print(f"❌ Error cargando historial: {str(e)}")
        return None

# Ejecutar
df_eventos = cargar_historial_eventos_blindado(archivo_historial)


🔄 Cargando historial de eventos de mantenimiento (FORZADO)...
📊 Dimensiones crudas: (9, 3)
   📅 Columna de fecha detectada: 0 (8 fechas válidas)
✅ Historial procesado exitosamente
   📅 Eventos válidos: 8
   📋 Columnas: ['fecha_evento', 'descripcion_falla', 'equipo']
   🔍 Ejemplo: 2023-03-15 00:00:00 - Falla: Fuga de Refrigerante


In [8]:
# Validación de compatibilidad temporal entre datasets - CORREGIDO
print("\n🔍 Validando compatibilidad temporal entre datasets...")

# Inicializar variables de control
eventos_utilizables = None
overlap_duration = None
compatibilidad_temporal = False

try:
    # Validar pre-requisitos
    if df is None or df.empty:
        print("   ❌ Dataset principal no disponible")
    elif df_eventos is None or df_eventos.empty:
        print("   ❌ Dataset de eventos no disponible")
    elif not isinstance(df.index, pd.DatetimeIndex):
        print("   ❌ Índice del dataset principal no es de tipo fecha")
    elif 'fecha_evento' not in df_eventos.columns:
        print("   ❌ No se encontró columna 'fecha_evento' en el historial")
    else:
        # Análisis de rangos temporales
        sensor_start, sensor_end = df.index.min(), df.index.max()
        eventos_limpios = df_eventos['fecha_evento'].dropna()

        if eventos_limpios.empty:
            print("   ❌ No hay eventos con fechas válidas")
        else:
            eventos_start, eventos_end = eventos_limpios.min(), eventos_limpios.max()

            print(f"📊 Análisis de cobertura temporal:")
            print(f"   🔧 Datos de sensores: {sensor_start} a {sensor_end}")
            print(f"   📅 Eventos registrados: {eventos_start} a {eventos_end}")

            # Calcular solapamiento temporal
            overlap_start = max(sensor_start, eventos_start)
            overlap_end = min(sensor_end, eventos_end)

            if overlap_start <= overlap_end:
                overlap_duration = overlap_end - overlap_start
                print(f"   ✅ Solapamiento detectado: {overlap_start} a {overlap_end}")
                print(f"   ⏱️  Duración del solapamiento: {overlap_duration}")

                # Eventos que caen dentro del rango de datos de sensores
                eventos_utilizables = eventos_limpios[
                    (eventos_limpios >= sensor_start) & (eventos_limpios <= sensor_end)
                ]

                print(f"   🎯 Eventos utilizables para entrenamiento: {len(eventos_utilizables)}")

                if len(eventos_utilizables) > 0:
                    compatibilidad_temporal = True
                    print(f"   ✅ COMPATIBILIDAD TEMPORAL CONFIRMADA")

                    # Análisis de distribución temporal de eventos
                    eventos_por_year = eventos_utilizables.dt.year.value_counts().sort_index()
                    print(f"   📈 Distribución anual de eventos: {dict(eventos_por_year)}")

                    # Estadísticas de densidad de eventos
                    dias_totales = (sensor_end - sensor_start).days
                    densidad_eventos = len(eventos_utilizables) / dias_totales if dias_totales > 0 else 0
                    print(f"   📊 Densidad de eventos: {densidad_eventos:.4f} eventos/día")
                else:
                    print(f"   ⚠️  Sin eventos utilizables dentro del rango de sensores")
            else:
                print(f"   ❌ No hay solapamiento temporal entre datasets")
                print(f"       Sensores terminan: {sensor_end}")
                print(f"       Eventos inician: {eventos_start}")

                # Análisis de eventos fuera de rango
                eventos_antes = eventos_limpios[eventos_limpios < sensor_start]
                eventos_despues = eventos_limpios[eventos_limpios > sensor_end]
                print(f"       Eventos antes del rango: {len(eventos_antes)}")
                print(f"       Eventos después del rango: {len(eventos_despues)}")

except Exception as e:
    print(f"   ❌ Error en validación temporal: {str(e)}")
    compatibilidad_temporal = False

# Resumen ejecutivo de la validación
print(f"\n📋 RESUMEN DE VALIDACIÓN TEMPORAL:")
print(f"   Compatibilidad temporal: {'✅ CONFIRMADA' if compatibilidad_temporal else '❌ NO CONFIRMADA'}")
print(f"   Eventos utilizables: {len(eventos_utilizables) if eventos_utilizables is not None else 0}")
print(f"   Duración de solapamiento: {overlap_duration if overlap_duration else 'N/A'}")

# Configurar variables para las siguientes etapas
if not compatibilidad_temporal:
    print(f"\n⚠️  ADVERTENCIA: Sin compatibilidad temporal, se procederá con análisis limitado")
    eventos_utilizables = pd.Series(dtype='datetime64[ns]')  # Serie vacía
else:
    print(f"\n🎯 Listo para proceder con ingeniería de características")


🔍 Validando compatibilidad temporal entre datasets...
📊 Análisis de cobertura temporal:
   🔧 Datos de sensores: 2023-01-01 00:00:00 a 2025-04-30 00:00:00
   📅 Eventos registrados: 2023-03-15 00:00:00 a 2025-03-01 00:00:00
   ✅ Solapamiento detectado: 2023-03-15 00:00:00 a 2025-03-01 00:00:00
   ⏱️  Duración del solapamiento: 717 days 00:00:00
   🎯 Eventos utilizables para entrenamiento: 8
   ✅ COMPATIBILIDAD TEMPORAL CONFIRMADA
   📈 Distribución anual de eventos: {2023: np.int64(4), 2024: np.int64(3), 2025: np.int64(1)}
   📊 Densidad de eventos: 0.0094 eventos/día

📋 RESUMEN DE VALIDACIÓN TEMPORAL:
   Compatibilidad temporal: ✅ CONFIRMADA
   Eventos utilizables: 8
   Duración de solapamiento: 717 days 00:00:00

🎯 Listo para proceder con ingeniería de características


## 2. 🏗️ Ingeniería de Características Temporales

### 📈 Características de Ventanas Móviles (Rolling Features)

Las características de ventanas móviles son fundamentales para capturar la **dinámica temporal del deterioro** en equipos industriales. Implementaremos múltiples horizontes temporales para detectar tanto degradaciones lentas como cambios súbitos.

In [9]:
# Función para crear características de ventanas móviles optimizada
def crear_rolling_features(df, ventanas=['6H', '24H', '72H'],
                          estadisticas=['mean', 'std', 'min', 'max'],
                          variables_prioritarias=None, max_features=200):
    """
    Crea características de ventanas móviles para múltiples estadísticas

    Parámetros:
    - df: DataFrame con series temporales
    - ventanas: Lista de ventanas temporales (ej: ['6H', '24H'])
    - estadisticas: Lista de estadísticas a calcular
    - variables_prioritarias: Variables específicas a procesar
    - max_features: Límite máximo de features a crear
    """
    print("📈 Creando características de ventanas móviles...")

    if variables_prioritarias is None:
        # Seleccionar automáticamente variables numéricas
        variables_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
        variables_prioritarias = variables_numericas[:20]  # Primeras 20

    print(f"   📊 Variables a procesar: {len(variables_prioritarias)}")
    print(f"   ⏱️  Ventanas temporales: {ventanas}")
    print(f"   📋 Estadísticas: {estadisticas}")

    rolling_features = pd.DataFrame(index=df.index)
    contador_features = 0

    for ventana in ventanas:
        print(f"\n   🔄 Procesando ventana {ventana}...")

        for variable in variables_prioritarias:
            if contador_features >= max_features:
                print(f"   ⚠️  Límite de {max_features} features alcanzado")
                break

            try:
                # Crear rolling window
                rolling = df[variable].rolling(window=ventana, min_periods=1)

                for stat in estadisticas:
                    if contador_features >= max_features:
                        break

                    nombre_feature = f"{variable}_roll_{ventana}_{stat}"

                    if stat == 'mean':
                        rolling_features[nombre_feature] = rolling.mean()
                    elif stat == 'std':
                        rolling_features[nombre_feature] = rolling.std()
                    elif stat == 'min':
                        rolling_features[nombre_feature] = rolling.min()
                    elif stat == 'max':
                        rolling_features[nombre_feature] = rolling.max()
                    elif stat == 'median':
                        rolling_features[nombre_feature] = rolling.median()
                    elif stat == 'skew':
                        rolling_features[nombre_feature] = rolling.skew()
                    elif stat == 'kurt':
                        rolling_features[nombre_feature] = rolling.kurt()

                    contador_features += 1

            except Exception as e:
                print(f"      ⚠️  Error con {variable}: {str(e)}")
                continue

        if contador_features >= max_features:
            break

    print(f"✅ Rolling features creadas: {contador_features}")
    print(f"📐 Dimensiones: {rolling_features.shape[0]:,} × {rolling_features.shape[1]}")

    return rolling_features

# Ejecutar creación de rolling features
print("\n🏗️ INICIANDO CREACIÓN DE CARACTERÍSTICAS TEMPORALES")
print("=" * 60)

try:
    rolling_features = crear_rolling_features(
        df=df,
        ventanas=['6H', '24H', '72H'],
        estadisticas=['mean', 'std', 'min', 'max'],
        max_features=150
    )

    print(f"\n💾 Memoria rolling features: {rolling_features.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

except Exception as e:
    print(f"❌ Error creando rolling features: {str(e)}")
    rolling_features = pd.DataFrame(index=df.index)  # DataFrame vacío como fallback


🏗️ INICIANDO CREACIÓN DE CARACTERÍSTICAS TEMPORALES
📈 Creando características de ventanas móviles...
   📊 Variables a procesar: 8
   ⏱️  Ventanas temporales: ['6H', '24H', '72H']
   📋 Estadísticas: ['mean', 'std', 'min', 'max']

   🔄 Procesando ventana 6H...

   🔄 Procesando ventana 24H...

   🔄 Procesando ventana 72H...
✅ Rolling features creadas: 96
📐 Dimensiones: 20,401 × 96

💾 Memoria rolling features: 15.1 MB


### ⏪ Características de Lag (Retrasos Temporales)

Los features de lag son esenciales para **modelar la memoria temporal** del sistema, permitiendo al modelo acceder a estados históricos del equipo para predecir comportamientos futuros.

In [10]:
# Función para crear características de lag CORREGIDA
def crear_lag_features(df, lags=['2H', '12H', '48H'],
                      variables_prioritarias=None, max_features=100):
    """
    Crea características de lag (retrasos temporales) con manejo robusto de errores

    CORRECCIÓN: Calcula períodos enteros basándose en la frecuencia del índice
    para evitar el error "unit abbreviation w/o a number"
    """
    print("⏪ Creando características de lag (retrasos temporales) - VERSIÓN CORREGIDA...")

    if variables_prioritarias is None:
        variables_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
        variables_prioritarias = variables_numericas[:12]  # Primeras 12

    print(f"   📊 Variables a procesar: {len(variables_prioritarias)}")
    print(f"   ⏱️  Lags temporales: {lags}")

    # Detectar frecuencia del índice
    print(f"   🔍 Analizando frecuencia temporal del dataset...")
    freq_detectada = pd.infer_freq(df.index)

    # Calcular diferencia temporal promedio como fallback
    if len(df.index) > 1:
        time_diffs = df.index[1:] - df.index[:-1]
        freq_promedio = time_diffs.median()
        print(f"   ⏱️  Frecuencia detectada: {freq_detectada}")
        print(f"   ⏱️  Frecuencia promedio: {freq_promedio}")
    else:
        freq_promedio = pd.Timedelta('1H')  # Default
        print(f"   ⚠️  Usando frecuencia por defecto: {freq_promedio}")

    lag_features = pd.DataFrame(index=df.index)
    contador_features = 0

    for lag_str in lags:
        print(f"\n   🔄 Procesando lag {lag_str}...")

        try:
            # CORRECCIÓN PRINCIPAL: Convertir lag string a timedelta y calcular períodos
            lag_timedelta = pd.Timedelta(lag_str)

            # Método 1: Usar frecuencia detectada
            if freq_detectada:
                try:
                    freq_td = pd.Timedelta(freq_detectada)
                    lag_periods = int(lag_timedelta / freq_td)
                    print(f"      ✅ Método 1 exitoso: {lag_str} = {lag_periods} períodos (freq: {freq_detectada})")
                except:
                    lag_periods = int(lag_timedelta / freq_promedio)
                    print(f"      ⚠️  Método 1 falló, usando promedio: {lag_str} = {lag_periods} períodos")
            else:
                # Método 2: Usar frecuencia promedio
                lag_periods = int(lag_timedelta / freq_promedio)
                print(f"      ✅ Método 2: {lag_str} = {lag_periods} períodos (freq promedio)")

            # Validar que lag_periods sea razonable
            if lag_periods <= 0:
                print(f"      ❌ Períodos inválidos ({lag_periods}), saltando lag {lag_str}")
                continue
            elif lag_periods > len(df):
                print(f"      ⚠️  Períodos muy altos ({lag_periods} > {len(df)}), ajustando a {len(df)//4}")
                lag_periods = len(df) // 4

            # Crear lag features para cada variable
            features_creados_lag = 0
            for variable in variables_prioritarias:
                if contador_features >= max_features:
                    print(f"   ⚠️  Límite global de {max_features} features alcanzado")
                    break

                try:
                    # CORRECCIÓN: Usar shift() con períodos enteros
                    nombre_feature = f"{variable}_lag_{lag_str}"
                    lag_features[nombre_feature] = df[variable].shift(lag_periods)
                    contador_features += 1
                    features_creados_lag += 1

                except Exception as e:
                    print(f"      ⚠️  Error con {variable}: {str(e)}")
                    continue

            print(f"      ✅ Features creados para {lag_str}: {features_creados_lag}")

        except Exception as e:
            print(f"      ❌ Error procesando lag {lag_str}: {str(e)}")
            continue

        if contador_features >= max_features:
            break

    # Crear características de diferencias con manejo robusto
    print(f"\n   📊 Creando características de diferencias...")

    diferencias_creadas = 0
    for variable in variables_prioritarias[:8]:  # Limitar a primeras 8 variables
        if contador_features >= max_features:
            break

        try:
            # Diferencia simple (método más robusto)
            diff_periods_12h = max(1, int(pd.Timedelta('12H') / freq_promedio))
            diff_name = f"{variable}_diff_12H"
            lag_features[diff_name] = df[variable] - df[variable].shift(diff_periods_12h)

            # Cambio porcentual
            pct_periods_24h = max(1, int(pd.Timedelta('24H') / freq_promedio))
            pct_name = f"{variable}_pct_change_24H"

            # Usar método robusto para cambio porcentual
            previous_values = df[variable].shift(pct_periods_24h)
            current_values = df[variable]

            # Evitar división por cero
            lag_features[pct_name] = np.where(
                previous_values != 0,
                (current_values - previous_values) / np.abs(previous_values) * 100,
                0  # Cuando el valor anterior es 0
            )

            contador_features += 2
            diferencias_creadas += 2

        except Exception as e:
            print(f"      ⚠️  Error con diferencias de {variable}: {str(e)}")
            continue

    print(f"      ✅ Características de diferencias creadas: {diferencias_creadas}")

    # Limpieza final
    print(f"\n   🧹 Aplicando limpieza final...")
    features_antes = lag_features.shape[1]

    # Eliminar columnas con demasiados NaN
    threshold_nan = int(0.8 * len(lag_features))  # Permitir hasta 80% de NaN
    lag_features = lag_features.dropna(axis=1, thresh=threshold_nan)

    # Eliminar columnas constantes
    for col in lag_features.columns:
        if lag_features[col].nunique() <= 1:
            lag_features = lag_features.drop(col, axis=1)

    features_despues = lag_features.shape[1]
    if features_antes != features_despues:
        print(f"      🗑️  Eliminados {features_antes - features_despues} features problemáticos")

    print(f"✅ Lag features creadas exitosamente: {contador_features} features totales")
    print(f"📐 Dimensiones finales: {lag_features.shape[0]:,} × {lag_features.shape[1]}")

    # Información de calidad
    if lag_features.shape[1] > 0:
        completitud = (lag_features.count().sum() / (lag_features.shape[0] * lag_features.shape[1])) * 100
        print(f"📈 Completitud promedio: {completitud:.1f}%")
    else:
        print(f"⚠️  No se generaron lag features válidas")

    return lag_features

# Ejecutar creación de lag features CORREGIDA
print("\n⏪ INICIANDO CREACIÓN DE LAG FEATURES CORREGIDA")
print("=" * 60)

try:
    lag_features = crear_lag_features(
        df=df,
        lags=['2H', '12H', '48H'],
        max_features=75
    )

    if not lag_features.empty:
        print(f"\n💾 Memoria lag features: {lag_features.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
        print(f"🎯 LAG FEATURES CREADAS EXITOSAMENTE")
    else:
        print(f"\n⚠️  Dataset de lag features vacío - usando DataFrame básico como fallback")

except Exception as e:
    print(f"❌ Error creando lag features: {str(e)}")
    import traceback
    traceback.print_exc()
    lag_features = pd.DataFrame(index=df.index)  # DataFrame vacío como fallback
    print(f"⚠️  Usando fallback - DataFrame vacío")


⏪ INICIANDO CREACIÓN DE LAG FEATURES CORREGIDA
⏪ Creando características de lag (retrasos temporales) - VERSIÓN CORREGIDA...
   📊 Variables a procesar: 8
   ⏱️  Lags temporales: ['2H', '12H', '48H']
   🔍 Analizando frecuencia temporal del dataset...
   ⏱️  Frecuencia detectada: h
   ⏱️  Frecuencia promedio: 0 days 01:00:00

   🔄 Procesando lag 2H...
      ⚠️  Método 1 falló, usando promedio: 2H = 2 períodos
      ✅ Features creados para 2H: 8

   🔄 Procesando lag 12H...
      ⚠️  Método 1 falló, usando promedio: 12H = 12 períodos
      ✅ Features creados para 12H: 8

   🔄 Procesando lag 48H...
      ⚠️  Método 1 falló, usando promedio: 48H = 48 períodos
      ✅ Features creados para 48H: 8

   📊 Creando características de diferencias...
      ✅ Características de diferencias creadas: 16

   🧹 Aplicando limpieza final...
✅ Lag features creadas exitosamente: 40 features totales
📐 Dimensiones finales: 20,401 × 40
📈 Completitud promedio: 99.9%

💾 Memoria lag features: 6.4 MB
🎯 LAG FEATURE

## 3. 🎯 Consolidación y Preparación del Dataset Final

### 🔗 Integración de Características

En esta fase crítica consolidamos todas las características creadas en un dataset unificado, optimizado para el entrenamiento de modelos de Machine Learning.

In [11]:
# Consolidación final del dataset de características - SIN RECORTES
print("🔗 Consolidando dataset final de características (MODO COMPLETO)...")
print("=" * 60)

try:
    # 1. Seleccionar variables originales (SIN LÍMITE)
    # Antes: variables_numericas[:25] -> Ahora: Todas
    variables_numericas = df.select_dtypes(include=[np.number]).columns.tolist()
    variables_importantes = variables_numericas

    # Crear dataset base
    dataset_final = df[variables_importantes].copy()
    print(f"📊 Variables originales incluidas: {dataset_final.shape[1]} (Todas)")

    # 2. Agregar Rolling Features (SIN LÍMITE DE CANTIDAD)
    if not rolling_features.empty:
        # Filtrar rolling features con variabilidad significativa (para no meter basura)
        # Mantenemos el filtro de columnas vacías, pero quitamos el recorte [:80]
        rolling_features_filtered = rolling_features.dropna(axis=1, thresh=int(0.7 * len(rolling_features)))
        rolling_features_filtered = rolling_features_filtered.select_dtypes(include=[np.number])

        features_con_variacion = []
        for col in rolling_features_filtered.columns:
            if rolling_features_filtered[col].std() > 1e-6:
                features_con_variacion.append(col)

        # CAMBIO AQUÍ: Se eliminó [:80] para incluir todas
        rolling_features_final = rolling_features_filtered[features_con_variacion]
        dataset_final = pd.concat([dataset_final, rolling_features_final], axis=1)
        print(f"📈 Rolling features agregadas: {rolling_features_final.shape[1]} (Todas las válidas)")
    else:
        print(f"⚠️  No se agregaron rolling features (dataset vacío)")

    # 3. Agregar Lag Features (SIN LÍMITE DE CANTIDAD)
    if not lag_features.empty:
        # Mantenemos filtro de calidad básico, pero quitamos el recorte [:40]
        lag_features_filtered = lag_features.dropna(axis=1, thresh=int(0.5 * len(lag_features)))
        lag_features_filtered = lag_features_filtered.select_dtypes(include=[np.number])

        features_con_variacion = []
        for col in lag_features_filtered.columns:
            if lag_features_filtered[col].std() > 1e-6:
                features_con_variacion.append(col)

        # CAMBIO AQUÍ: Se eliminó [:40] para incluir todas
        lag_features_final = lag_features_filtered[features_con_variacion]
        dataset_final = pd.concat([dataset_final, lag_features_final], axis=1)
        print(f"⏪ Lag features agregadas: {lag_features_final.shape[1]} (Todas las válidas)")
    else:
        print(f"⚠️  No se agregaron lag features (dataset vacío)")

    print(f"\n🧹 Limpieza final del dataset...")

    # Eliminar columnas completamente nulas
    columnas_antes = dataset_final.shape[1]
    dataset_final = dataset_final.dropna(axis=1, how='all')

    # Eliminar features prácticamente constantes (varianza 0)
    # NOTA: Esto es necesario para que el modelo LSTM no falle, pero solo borra lo que es plano.
    for col in dataset_final.select_dtypes(include=[np.number]).columns:
        if dataset_final[col].std() < 1e-10:
            dataset_final = dataset_final.drop(col, axis=1)

    columnas_despues = dataset_final.shape[1]
    if columnas_antes != columnas_despues:
        print(f"   🗑️  Eliminadas {columnas_antes - columnas_despues} columnas constantes/vacías")

    # Convertir a float32 para optimizar memoria
    numeric_columns = dataset_final.select_dtypes(include=[np.number]).columns
    dataset_final[numeric_columns] = dataset_final[numeric_columns].astype(np.float32)

    # Información final del dataset
    print(f"\n📊 DATASET FINAL CONSOLIDADO:")
    print(f"   📐 Dimensiones: {dataset_final.shape[0]:,} filas × {dataset_final.shape[1]} columnas")
    print(f"   💾 Memoria: {dataset_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    # Verificar calidad del dataset final
    completitud_final = (dataset_final.count().sum() / (dataset_final.shape[0] * dataset_final.shape[1])) * 100
    print(f"\n✅ Calidad del dataset final: {completitud_final:.1f}%")

except Exception as e:
    print(f"❌ Error en consolidación del dataset: {str(e)}")
    import traceback
    traceback.print_exc()
    dataset_final = df.select_dtypes(include=[np.number]).copy()

🔗 Consolidando dataset final de características (MODO COMPLETO)...
📊 Variables originales incluidas: 8 (Todas)
📈 Rolling features agregadas: 96 (Todas las válidas)
⏪ Lag features agregadas: 40 (Todas las válidas)

🧹 Limpieza final del dataset...

📊 DATASET FINAL CONSOLIDADO:
   📐 Dimensiones: 20,401 filas × 144 columnas
   💾 Memoria: 11.4 MB

✅ Calidad del dataset final: 100.0%


## 3.5. Creación de Variable Objetivo

### Etiquetado de Fallas con Ventana de Pre-falla

Esta es la fase crítica donde convertimos los eventos de mantenimiento en la variable objetivo binaria 'falla'. Utilizamos una ventana de 7 días antes de cada evento para marcar las muestras como positivas (falla inminente).

In [12]:
# DEBUG Y CREACIÓN FORZADA DE VARIABLE OBJETIVO
print("🔍 DIAGNÓSTICO DEL PROBLEMA")
print("=" * 60)

# 1. VERIFICAR ESTADO ACTUAL DEL DATASET
print("1. Verificando estado actual del dataset:")
print(f"   Dimensiones: {dataset_final.shape}")
print(f"   Columnas disponibles: {len(dataset_final.columns)}")
print(f"   Variable 'falla' presente: {'SÍ' if 'falla' in dataset_final.columns else 'NO'}")
print(f"   Primeras 5 columnas: {list(dataset_final.columns[:5])}")
print(f"   Últimas 5 columnas: {list(dataset_final.columns[-5:])}")

# 2. VERIFICAR EVENTOS DISPONIBLES
print(f"\n2. Verificando eventos disponibles:")
if 'eventos_utilizables' in locals():
    print(f"   Variable eventos_utilizables existe: SÍ")
    print(f"   Tipo: {type(eventos_utilizables)}")
    print(f"   Cantidad de eventos: {len(eventos_utilizables) if eventos_utilizables is not None else 0}")
    if len(eventos_utilizables) > 0:
        print(f"   Primer evento: {eventos_utilizables.iloc[0]}")
        print(f"   Último evento: {eventos_utilizables.iloc[-1]}")
else:
    print(f"   Variable eventos_utilizables: NO EXISTE")

# 3. VERIFICAR COMPATIBILIDAD TEMPORAL
print(f"\n3. Verificando compatibilidad temporal:")
if 'compatibilidad_temporal' in locals():
    print(f"   Compatibilidad temporal: {compatibilidad_temporal}")
else:
    print(f"   Variable compatibilidad_temporal: NO DEFINIDA")

# 4. RECREAR EVENTOS UTILIZABLES SI ES NECESARIO
print(f"\n4. Recreando eventos utilizables...")
try:
    # Verificar si tenemos df_eventos
    if 'df_eventos' in locals() and df_eventos is not None:
        print(f"   Dataset de eventos disponible: SÍ ({len(df_eventos)} filas)")

        # Recrear eventos utilizables
        if 'fecha_evento' in df_eventos.columns:
            eventos_validos = df_eventos['fecha_evento'].dropna()
            print(f"   Eventos con fecha válida: {len(eventos_validos)}")

            # Filtrar eventos dentro del rango del dataset principal
            sensor_start, sensor_end = dataset_final.index.min(), dataset_final.index.max()
            eventos_en_rango = eventos_validos[
                (eventos_validos >= sensor_start) & (eventos_validos <= sensor_end)
            ]

            print(f"   Eventos en rango temporal: {len(eventos_en_rango)}")
            eventos_utilizables = eventos_en_rango

        else:
            print(f"   ERROR: Columna 'fecha_evento' no encontrada")
            eventos_utilizables = pd.Series(dtype='datetime64[ns]')
    else:
        print(f"   Dataset de eventos NO disponible")
        eventos_utilizables = pd.Series(dtype='datetime64[ns]')

except Exception as e:
    print(f"   ERROR recreando eventos: {str(e)}")
    eventos_utilizables = pd.Series(dtype='datetime64[ns]')

# 5. CREAR VARIABLE OBJETIVO DE FORMA FORZADA
print(f"\n5. Creando variable objetivo FORZADAMENTE...")

def crear_variable_falla_robusta(dataset, eventos, ventana_dias=7):
    """
    Versión robusta de creación de variable objetivo
    """
    print(f"   Iniciando creación con {len(eventos)} eventos...")

    # Crear columna 'falla' inicializada en 0
    dataset = dataset.copy()
    dataset['falla'] = 0

    if len(eventos) == 0:
        print(f"   ⚠️  Sin eventos disponibles - todas las muestras como normales")
        return dataset

    ventana_timedelta = pd.Timedelta(days=ventana_dias)
    muestras_marcadas = 0
    eventos_procesados = 0

    for fecha_evento in eventos:
        try:
            # Calcular ventana de pre-falla
            inicio_ventana = fecha_evento - ventana_timedelta
            fin_ventana = fecha_evento

            # Crear máscara para el período de pre-falla
            mask = (dataset.index >= inicio_ventana) & (dataset.index <= fin_ventana)
            muestras_en_ventana = mask.sum()

            if muestras_en_ventana > 0:
                dataset.loc[mask, 'falla'] = 1
                muestras_marcadas += muestras_en_ventana
                eventos_procesados += 1
                print(f"     ✅ Evento {eventos_procesados}: {fecha_evento} -> {muestras_en_ventana} muestras")
            else:
                print(f"     ⚠️  Evento {fecha_evento}: Sin muestras en ventana")

        except Exception as e:
            print(f"     ❌ Error procesando evento {fecha_evento}: {str(e)}")
            continue

    # Estadísticas finales
    total_muestras = len(dataset)
    muestras_positivas = (dataset['falla'] == 1).sum()
    muestras_negativas = (dataset['falla'] == 0).sum()
    porcentaje_positivas = (muestras_positivas / total_muestras) * 100

    print(f"\n   📊 RESULTADO DE CREACIÓN:")
    print(f"     🔴 Muestras positivas: {muestras_positivas:,} ({porcentaje_positivas:.2f}%)")
    print(f"     🔵 Muestras negativas: {muestras_negativas:,} ({100-porcentaje_positivas:.2f}%)")
    print(f"     ✅ Eventos procesados exitosamente: {eventos_procesados}/{len(eventos)}")

    return dataset

# Ejecutar creación robusta
try:
    print(f"   Estado antes: {dataset_final.shape}")
    dataset_final = crear_variable_falla_robusta(dataset_final, eventos_utilizables)
    print(f"   Estado después: {dataset_final.shape}")

    # VERIFICACIÓN CRÍTICA
    if 'falla' in dataset_final.columns:
        print(f"   ✅ ÉXITO: Variable 'falla' creada correctamente")
        distribución = dataset_final['falla'].value_counts().sort_index()
        print(f"   📊 Distribución: {dict(distribución)}")
        print(f"   🎯 Columna 'falla' confirmada en posición: {list(dataset_final.columns).index('falla')}")
    else:
        print(f"   ❌ ERROR: Variable 'falla' AÚN no se creó")
        # Crear manualmente como último recurso
        print(f"   🚨 CREACIÓN MANUAL DE EMERGENCIA...")
        dataset_final['falla'] = 0
        print(f"   ✅ Variable 'falla' creada manualmente (todas negativas)")

except Exception as e:
    print(f"   ❌ ERROR en creación robusta: {str(e)}")
    # Crear variable por defecto
    dataset_final['falla'] = 0
    print(f"   🚨 Variable por defecto creada")

# 6. VERIFICACIÓN FINAL ANTES DEL GUARDADO
print(f"\n6. VERIFICACIÓN FINAL:")
print(f"   ✅ Dimensiones finales: {dataset_final.shape}")
print(f"   ✅ Variable 'falla' presente: {'SÍ' if 'falla' in dataset_final.columns else 'NO'}")
print(f"   ✅ Últimas 3 columnas: {list(dataset_final.columns[-3:])}")

if 'falla' in dataset_final.columns:
    print(f"   ✅ Tipo de datos 'falla': {dataset_final['falla'].dtype}")
    print(f"   ✅ Valores únicos: {sorted(dataset_final['falla'].unique())}")
    print(f"   ✅ Conteo de valores: {dict(dataset_final['falla'].value_counts())}")
    print(f"\n🎉 VARIABLE OBJETIVO CONFIRMADA - LISTO PARA GUARDADO")
else:
    print(f"\n❌ VARIABLE OBJETIVO FALLA - REVISAR MANUALMENTE")

print("\n" + "="*60)

🔍 DIAGNÓSTICO DEL PROBLEMA
1. Verificando estado actual del dataset:
   Dimensiones: (20401, 144)
   Columnas disponibles: 144
   Variable 'falla' presente: NO
   Primeras 5 columnas: ['pres_aceite_comp', 'rpm', 'presion_aceite_motor', 'presion_agua', 'presion_carter']
   Últimas 5 columnas: ['temp_aceite_motor_pct_change_24H', 'temp_agua_motor_diff_12H', 'temp_agua_motor_pct_change_24H', 'temp_mult_adm_izq_diff_12H', 'temp_mult_adm_izq_pct_change_24H']

2. Verificando eventos disponibles:
   Variable eventos_utilizables existe: SÍ
   Tipo: <class 'pandas.core.series.Series'>
   Cantidad de eventos: 8
   Primer evento: 2023-03-15 00:00:00
   Último evento: 2025-03-01 00:00:00

3. Verificando compatibilidad temporal:
   Compatibilidad temporal: True

4. Recreando eventos utilizables...
   Dataset de eventos disponible: SÍ (8 filas)
   Eventos con fecha válida: 8
   Eventos en rango temporal: 8

5. Creando variable objetivo FORZADAMENTE...
   Estado antes: (20401, 144)
   Iniciando creac

## 4. 💾 Guardado y Finalización

### 📁 Persistencia del Dataset de Características

Guardamos el dataset final optimizado en múltiples formatos para garantizar compatibilidad y eficiencia en las fases posteriores de modelado.

In [13]:
# Guardado del dataset final con metadatos completos
print("💾 Guardando dataset final de características...")
print("=" * 60)

# Configurar rutas de salida
output_dir = ruta_processed
output_dir.mkdir(parents=True, exist_ok=True)

# Nombres de archivos de salida
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
base_name = f'features_dataset_{timestamp}'

try:
    # 1. Guardar en formato Parquet (optimizado)
    archivo_parquet = output_dir / f'{base_name}.parquet'
    dataset_final.to_parquet(archivo_parquet, engine='pyarrow', compression='snappy')
    tamaño_parquet = archivo_parquet.stat().st_size / (1024 * 1024)
    print(f"✅ Parquet guardado: {archivo_parquet.name} ({tamaño_parquet:.1f} MB)")

    # 2. Guardar en formato CSV (compatibilidad)
    archivo_csv = output_dir / f'{base_name}.csv'
    dataset_final.to_csv(archivo_csv, encoding='utf-8')
    tamaño_csv = archivo_csv.stat().st_size / (1024 * 1024)
    print(f"✅ CSV guardado: {archivo_csv.name} ({tamaño_csv:.1f} MB)")

    # 3. Generar archivo de metadatos
    archivo_metadata = output_dir / f'{base_name}_metadata.txt'
    with open(archivo_metadata, 'w', encoding='utf-8') as f:
        f.write(f"METADATOS DEL DATASET DE CARACTERÍSTICAS\n")
        f.write(f"=" * 50 + "\n\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Notebook: 03_feature_engineering_b.ipynb\n")
        f.write(f"\nDimensiones: {dataset_final.shape[0]:,} × {dataset_final.shape[1]}\n")
        f.write(f"Período temporal: {dataset_final.index.min()} → {dataset_final.index.max()}\n")
        f.write(f"Memoria utilizada: {dataset_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB\n")
        f.write(f"\nTipos de características:\n")
        f.write(f"  - Variables originales: {len(variables_importantes)}\n")
        if 'rolling_features_final' in locals():
            f.write(f"  - Rolling features: {rolling_features_final.shape[1]}\n")
        if 'lag_features_final' in locals():
            f.write(f"  - Lag features: {lag_features_final.shape[1]}\n")
        f.write(f"\nEventos de mantenimiento:\n")
        if eventos_utilizables is not None and len(eventos_utilizables) > 0:
            f.write(f"  - Eventos utilizables: {len(eventos_utilizables)}\n")
            f.write(f"  - Compatibilidad temporal: {'Confirmada' if compatibilidad_temporal else 'No confirmada'}\n")
        else:
            f.write(f"  - Sin eventos utilizables identificados\n")

        f.write(f"\nLista de columnas:\n")
        for i, col in enumerate(dataset_final.columns, 1):
            f.write(f"  {i:3d}. {col}\n")

    print(f"✅ Metadatos guardados: {archivo_metadata.name}")

    # 4. Generar resumen estadístico
    archivo_stats = output_dir / f'{base_name}_statistics.csv'
    stats_desc = dataset_final.describe()
    stats_desc.to_csv(archivo_stats, encoding='utf-8')
    print(f"✅ Estadísticas guardadas: {archivo_stats.name}")

    # Resumen final
    print(f"\n🎯 FEATURE ENGINEERING COMPLETADO EXITOSAMENTE")
    print(f"📁 Archivos generados en {output_dir}:")
    print(f"   📦 {base_name}.parquet - Dataset principal (comprimido)")
    print(f"   📄 {base_name}.csv - Dataset principal (CSV)")
    print(f"   📋 {base_name}_metadata.txt - Metadatos completos")
    print(f"   📊 {base_name}_statistics.csv - Estadísticas descriptivas")

    print(f"🎉 Ingeniería de Características finalizado exitosamente!")

except Exception as e:
    print(f"❌ Error al guardar dataset: {str(e)}")
    import traceback
    traceback.print_exc()

    # Guardado de emergencia
    backup_file = output_dir / f'features_dataset_emergency_backup.csv'
    dataset_final.to_csv(backup_file)
    print(f"💾 Guardado de emergencia: {backup_file}")

💾 Guardando dataset final de características...
✅ Parquet guardado: features_dataset_20251210_000020.parquet (12.2 MB)
✅ CSV guardado: features_dataset_20251210_000020.csv (28.1 MB)
✅ Metadatos guardados: features_dataset_20251210_000020_metadata.txt
✅ Estadísticas guardadas: features_dataset_20251210_000020_statistics.csv

🎯 FEATURE ENGINEERING COMPLETADO EXITOSAMENTE
📁 Archivos generados en data/processed:
   📦 features_dataset_20251210_000020.parquet - Dataset principal (comprimido)
   📄 features_dataset_20251210_000020.csv - Dataset principal (CSV)
   📋 features_dataset_20251210_000020_metadata.txt - Metadatos completos
   📊 features_dataset_20251210_000020_statistics.csv - Estadísticas descriptivas
🎉 Ingeniería de Características finalizado exitosamente!


In [14]:
# -------------------------------------------------------------------------
# CORRECCIÓN DE NOMBRE PARA EL SIGUIENTE NOTEBOOK
# -------------------------------------------------------------------------
import shutil

# 1. Definir la ruta exacta que busca el Notebook 04
archivo_requerido_parquet = ruta_processed / 'featured_dataset_with_target.parquet'
archivo_requerido_csv = ruta_processed / 'featured_dataset_with_target.csv'

print(f"🔧 Guardando copias estandarizadas para el Notebook 04...")

try:
    # Opción A: Guardar directamente desde el DataFrame en memoria (Más seguro)
    if 'dataset_final' in locals():
        dataset_final.to_parquet(archivo_requerido_parquet, engine='pyarrow', compression='snappy')
        dataset_final.to_csv(archivo_requerido_csv)
        print(f"   ✅ Archivo creado: {archivo_requerido_parquet}")
        print(f"   ✅ Archivo creado: {archivo_requerido_csv}")

    # Opción B: Si ya cerraste pandas, renombramos el último generado
    else:
        # (Esto solo se ejecuta si dataset_final no está en memoria)
        print("   ⚠️ DataFrame no encontrado en memoria, buscando último archivo generado...")
        # Lógica de respaldo no necesaria si acabas de correr todo.
        pass

    print("\n🚀 LISTO: Ahora el Notebook 04 encontrará el archivo 'featured_dataset_with_target.parquet' sin problemas.")

except Exception as e:
    print(f"❌ Error al guardar el archivo estandarizado: {str(e)}")

🔧 Guardando copias estandarizadas para el Notebook 04...
   ✅ Archivo creado: data/processed/featured_dataset_with_target.parquet
   ✅ Archivo creado: data/processed/featured_dataset_with_target.csv

🚀 LISTO: Ahora el Notebook 04 encontrará el archivo 'featured_dataset_with_target.parquet' sin problemas.


## 📋 Resumen del Feature Engineering

### 🎯 Logros Alcanzados

1. **✅ Carga de datos exitosa** - Integración de series temporales y historial de eventos
2. **✅ Validación temporal** - Confirmación de compatibilidad entre datasets
3. **✅ Características temporales** - Creación de rolling features y lag features
4. **✅ Dataset optimizado** - Consolidación y optimización de memoria
5. **✅ Persistencia completa** - Guardado en múltiples formatos con metadatos
